# input processing for running EpiBERT on sample data
EpiBERT requires (1) an input motif enrichment file generated using the Simple Enrichment Analysis (SEA) tool from the MEME-suite and (2) a processed ATAC-seq read ends file

You will need working installations of bedtools, samtools, and MEME to process the data as described below.  

In [2]:
# We will start by creating the processed motif enrichment file for an input dataset
# as an example to start we can download IDR thresholded peaks for K562
!wget -q https://www.encodeproject.org/files/ENCFF135AEX/@@download/ENCFF135AEX.bed.gz

In [3]:
# ensure that you have hg38 fasta downloaded (can get from here https://www.encodeproject.org/references/ENCSR938RZZ/)
# ensure that the fasta file is indexed
!samtools faidx hg38_erccpatch.fa

In [3]:
#now we will take the top 50000 peaks and extract the 128 bp around the peak center
!zcat ENCFF135AEX.bed.gz | sort -k5,5nr | head -n 50000 | awk '{OFS = "\t" }{print $1,$2+$10-64,$2+$10+64}' | sort -k1,1 -k2,2n > ENCFF135AEX.sorted.peaks.bed
!bedtools getfasta -fi hg38_erccpatch.fa -bed ENCFF135AEX.sorted.peaks.bed > ENCFF135AEX.peaks.fasta
!bedtools getfasta -fi hg38_erccpatch.fa -bed all_peaks_merged.counts.shared.centered.bed > bg_peaks.fasta

sort: write failed: 'standard output': Broken pipe
sort: write error


In [3]:
#run simple enrichment analysis from MEME using the consensus PWMs from https://resources.altius.org/~jvierstra/projects/motif-clustering-v2.0beta/
!/home/jupyter/meme-5.5.6/src/sea --p ENCFF135AEX.peaks.fasta --m consensus_pwms.meme --n bg_peaks.fasta --thresh 1.0 --verbosity 1
!mv sea_out/sea.tsv ENCFF135AEX.motifs.tsv

In [4]:
# Next we prepare the BAM file
# we can download the processed BAM file from encode for K562 (https://www.encodeproject.org/files/ENCFF534DCE/@@download/ENCFF534DCE.bam)

In [ ]:
# first we sort the bam
!samtools sort -n ENCFF534DCE.bam -o ENCFF534DCE.sorted.bam
# then for each pair extract the 5 and 3' cut sites and make the required Tn5 adjustmenet
!bedtools bamtobed -i ENCFF534DCE.sorted.bam -bedpe | awk '$1 == $4' | awk '$8 >= 20' | awk -v OFS="\t" '{if($9 == "+"){print $1,$2+4,$6-5}else if($9=="-"){print $1,$5+4,$3-5}}' | awk -v OFS="\t" '{if($3<$2){print $1,$3,$2}else if($3>$2){print $1,$2,$3}}' | awk '$3-$2 > 0' | gzip > K562.bed.gz

[bam_sort_core] merging from 30 files and 1 in-memory blocks...
